In [16]:
import psycopg2
import pandas as pd
from datetime import datetime
import pytz
from rapidfuzz import fuzz, process
from datetime import datetime, time
import uuid

In [17]:
df = pd.read_csv('databaseUpdated2024-12-07 (1).csv')
df.head()

,fight,redCorner,blueCorner,winner,event,referee,method_of_victory,date,venue,title_fight,...,redCorner_subs_attempted,blueCorner_subs_attempted,round,time,redCorner_height,blueCorner_height,redCorner_reach,blueCorner_reach,redCorner_stance,blueCorner_stance
0,Petr Yan vs Deiveson Figueiredo,Petr Yan,Deiveson Figueiredo,Petr Yan,UFC Fight Night: Yan vs. Figueiredo,Mike Beltran,Decision - Unanimous,11.23.2024,Galaxy Arena,no,...,0,0,5,5:00,"5' 7""","5' 5""","67""","68""",Switch,Orthodox
1,Xiaonan Yan vs Tabatha Ricci,Yan Xiaonan,Tabatha Ricci,Yan Xiaonan,UFC Fight Night: Yan vs. Figueiredo,Marc Goddard,Decision - Unanimous,11.23.2024,Galaxy Arena,no,...,0,0,3,5:00,"5' 5""","5' 1""","63""","61""",Orthodox,Orthodox
2,Kenan Song vs Muslim Salikhov,Song Kenan,Muslim Salikhov,Muslim Salikhov,UFC Fight Night: Yan vs. Figueiredo,Mark Craig,KO/TKO,11.23.2024,Galaxy Arena,no,...,0,0,1,3:49,"6' 0""","5' 11""","71""","70""",Orthodox,Orthodox
3,Cong Wang vs Gabriella Fernandes,Wang Cong,Gabriella Fernandes,Gabriella Fernandes,UFC Fight Night: Yan vs. Figueiredo,Marc Goddard,Submission,11.23.2024,Galaxy Arena,no,...,0,1,2,3:49,"5' 6""","5' 6""","66""","66""",Southpaw,Southpaw
4,Volkan Oezdemir vs Carlos Ulberg,Volkan Oezdemir,Carlos Ulberg,Carlos Ulberg,UFC Fight Night: Yan vs. Figueiredo,Mike Beltran,Decision - Unanimous,11.23.2024,Galaxy Arena,no,...,0,0,3,5:00,"6' 2""","6' 4""","75""","77""",Orthodox,Orthodox


In [18]:
print(df.columns)

Index(['fight', 'redCorner', 'blueCorner', 'winner', 'event', 'referee',
       'method_of_victory', 'date', 'venue', 'title_fight', 'billing',
       'redCorner_wins', 'blueCorner_wins', 'redCorner_losses',
       'blueCorner_losses', 'redCorner_draws', 'blueCorner_draws',
       'redCorner_age', 'blueCorner_age', 'redCorner_nation',
       'blueCorner_nation', 'redCorner_fan', 'blueCorner_fan',
       'redCorner_knockdowns', 'blueCorner_knockdowns', 'redCorner_sig_str',
       'blueCorner_sig_str', 'redCorner_sig_str_percentage',
       'blueCorner_sig_str_percentage', 'redCorner_total_str',
       'blueCorner_total_str', 'redCorner_takedowns', 'blueCorner_takedowns',
       'redCorner_takedown_percentage', 'blueCorner_takedown_percentage',
       'redCorner_subs_attempted', 'blueCorner_subs_attempted', 'round',
       'time', 'redCorner_height', 'blueCorner_height', 'redCorner_reach',
       'blueCorner_reach', 'redCorner_stance', 'blueCorner_stance'],
      dtype='object')


In [19]:
#Dropping all rows where red and blue corners are not recorded correctly
#Not recorded correctly until 03.21.2010
#Since this will affect the accuracy of the model I am removing it for now
#Will try and find a credible source to fix this at a later date

rows, columns = df.shape

event_condition = df['event'] == 'UFC 110: Nogueira vs Velasquez'
fight_condition = df['fight'] == 'Cain Velasquez vs Antonio Rodrigo Nogueira'

recordedIncorrectly = df.loc[event_condition & fight_condition].index[0]
recordedIncorrectly -= rows
print(recordedIncorrectly)
df = df.iloc[:recordedIncorrectly]

-1306


In [20]:
#output should be Jason Brilz vs Eric Schafer
lengthDF = len(df)
print(lengthDF)

print(df.iloc[-1])

6619
fight                             Jason Brilz vs Eric Schafer
redCorner                                        Eric Schafer
blueCorner                                        Jason Brilz
winner                                            Jason Brilz
event                                 UFC Live: Vera vs Jones
referee                                           Tom Johnson
method_of_victory                        Decision - Unanimous
date                                               03.21.2010
venue                                      The Odeum Colorado
title_fight                                                no
billing                                      Preliminary Card
redCorner_wins                                             11
blueCorner_wins                                            17
redCorner_losses                                            4
blueCorner_losses                                           2
redCorner_draws                                             2
blu

In [21]:
'''spring:
  application:
    name: project-database


  datasource:
    url: jdbc:postgresql://localhost:5432/project_database
    username: postgres
    password: postgres_123
    driver-class-name: org.postgresql.Driver '''


#Initalize sport id
conn = psycopg2.connect(
    dbname="project_database",
    user="postgres",
    password="postgres_123",
    host="localhost", 
    port="5432"
)
cur = conn.cursor()

sport = 'mixed_martial_arts_ufc'
sport_name = 'UFC'
cur.execute("INSERT INTO sport(id, name) VALUES (%s, %s)", (sport, sport_name))
conn.commit()



In [22]:
new_df = df[['event', 'date']]
new_df.head()

,event,date
0,UFC Fight Night: Yan vs. Figueiredo,11.23.2024
1,UFC Fight Night: Yan vs. Figueiredo,11.23.2024
2,UFC Fight Night: Yan vs. Figueiredo,11.23.2024
3,UFC Fight Night: Yan vs. Figueiredo,11.23.2024
4,UFC Fight Night: Yan vs. Figueiredo,11.23.2024


In [23]:
for index, row in df.iterrows():
    try:
        event_name = row['event']
        start_date = row['date']
        parsed_date = datetime.strptime(start_date, '%m.%d.%Y')
        start_date = datetime.combine(parsed_date, time.min).isoformat() + 'Z'
        event_id = str(uuid.uuid4())

        cur.execute("""
            INSERT INTO event(id, name, start_date, sport_id)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (name, start_date) DO NOTHING;
        """, (event_id, event_name, start_date, sport))

    except Exception as e:
        print(f"Error on row {index}: {e}")
        conn.rollback()
conn.commit()
cur.close()
conn.close()
